In [145]:
import pandas as pd
import numpy as np

master = pd.read_csv('../Files/COLLECTIONS/STORY MASTER.csv')
po_dump = pd.read_csv('../Files/COLLECTIONS/hrx-po-fresh.csv')
sales = pd.read_csv('../Files/COLLECTIONS/SALES 15DEC24-09FEB25.CSV')
sales = sales[ (sales['master_brand'] == "HRX by Hrithik Roshan" ) & (sales['master_category'] == "Apparel" ) & 
                ((sales['business_unit'] == "Men's Sports Apparel" ) | (sales['business_unit'] == "Women's Sports Apparel")) &
                ((sales['gender'] == "Men" ) | (sales['gender'] == "Women"))]
inventory_snapshot = pd.read_csv('../Files/COLLECTIONS/Inventory_snapshot_20250210.csv')
inventory_snapshot = inventory_snapshot[ (inventory_snapshot['master_brand'] == "HRX by Hrithik Roshan" ) & (inventory_snapshot['master_category'] == "Apparel" ) & 
                ((inventory_snapshot['business_unit'] == "Men's Sports Apparel" ) | (inventory_snapshot['business_unit'] == "Women's Sports Apparel")) &
                ((inventory_snapshot['gender'] == "Men" ) | (inventory_snapshot['gender'] == "Women"))]


In [146]:
#functions
def getSalesAmount(sid):
    dump = sales[sales['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        sales_value = dump['revenue'].sum()
        return sales_value

def getTaxAmount(sid):
    dump = sales[sales['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        tax_value = dump['tax'].sum()
        return tax_value

def getRoyaltyAmount(sid):
    dump = sales[sales['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        royalty_value = dump['royalty'].sum()
        return royalty_value

def getPurchasePriceAmount(sid):
    dump = sales[sales['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        purchase_price_value = dump['purchase_price'].sum()
        return purchase_price_value

def getTotalMRPAmount(sid):
    dump = sales[sales['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        tmrp_value = dump['tot_mrp'].sum()
        return tmrp_value

def getInventory(sid):
    dump = inventory_snapshot[inventory_snapshot['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        qty = dump['units'].sum()
        return qty

def getSalesQuantity(sid):
    dump = sales[sales['style_id'] == sid]
    if len(dump) == 0:
        return 0
    else:
        sales_value = dump['quantity'].sum()
        return sales_value

def getStory(sid):
    dump = master[master['style_id'] == sid]
    if len(dump) == 0:
        return '-'
    else:
        story_name = dump.iloc[0]['story']
        return story_name

def getSports(van):
    if '_KL' in van:
        return 'Lifestyle'
    elif '_KR' in van:
        return 'Running'
    elif '_KY' in van:
        return 'Yoga'    
    elif '_KT' in van:
        return 'Training'
    elif '_KO' in van:
        return 'Outdoor'
    else:
        return '-'

def getPOOptions(sports,story,gender,article):
    dump = po_dump[ (po_dump['sports'] == sports) & (po_dump['story'] == story) & (po_dump['gender'] == gender) & 
                    (po_dump['article_type'] == article) ]
    if len(dump) == 0:
        return 0
    else:
        count = len(dump['style_id'].unique())
        return count

def getGRNOptions(sports,story,gender,article):
    dump = po_dump[ (po_dump['sports'] == sports) & (po_dump['story'] == story) & (po_dump['gender'] == gender) & 
                    (po_dump['article_type'] == article) & (po_dump['grn_quantity'] != 0) ]
    if len(dump) == 0:
        return 0
    else:
        count = len(dump['style_id'].unique())
        return count

def getMRPBracket(sid):
    dump = master[master['style_id'] == sid]
    if len(dump) == 0:
        return '-'
    else:
        mrp_bkt = dump.iloc[0]['mrp_bracket']
        return mrp_bkt


master['GMV'] = master['style_id'].apply( getSalesAmount )
master['Tax'] = master['style_id'].apply( getTaxAmount )
master['Royalty'] = master['style_id'].apply( getRoyaltyAmount )
master['Purchase_Price'] = master['style_id'].apply( getPurchasePriceAmount )
master['Total_MRP'] = master['style_id'].apply( getTotalMRPAmount ) 
master['Inventory'] = master['style_id'].apply( getInventory  )                                             
master['Sold_Qty'] = master['style_id'].apply( getSalesQuantity )

po_dump['story'] = po_dump['style_id'].apply( getStory )
po_dump['sports'] = po_dump['vendor_article_number'].apply( getSports )
po_dump['mrp_bracket'] = po_dump['style_id'].apply( getMRPBracket )


In [147]:
po_dump = po_dump[ po_dump['Bulk / Catalogue'] == 'Bulk' ]
master_pivot = pd.pivot_table(master,index=['sports','story','gender','article_type'],values=['GMV','Tax','Royalty','Total_MRP','Purchase_Price','Inventory','Sold_Qty'],
                              aggfunc=np.sum)
master_pivot['GM%'] = (( master_pivot['GMV'] - master_pivot['Purchase_Price'] - master_pivot['Tax'] - master_pivot['Royalty'] ) / ( master_pivot['GMV'] ))
master_pivot['TD%'] = (( master_pivot['Total_MRP'] - master_pivot['GMV'] ) / ( master_pivot['Total_MRP'] ))
master_pivot = master_pivot.reset_index()
master_pivot['PO_Options'] = master_pivot.apply( lambda item :  getPOOptions(item['sports'],item['story'],item['gender'],item['article_type']), axis=1 )
master_pivot['GRN_Options'] = master_pivot.apply( lambda item :  getGRNOptions(item['sports'],item['story'],item['gender'],item['article_type']), axis=1 )

master_pivot


C:\Users\kunaljain.vc\AppData\Local\Temp\ipykernel_19952\4156678238.py:2: FutureWarning: The provided callable <function sum at 0x00000181D9C23D80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  master_pivot = pd.pivot_table(master,index=['sports','story','gender','article_type'],values=['GMV','Tax','Royalty','Total_MRP','Purchase_Price','Inventory','Sold_Qty'],


,sports,story,gender,article_type,GMV,Inventory,Purchase_Price,Royalty,Sold_Qty,Tax,Total_MRP,GM%,TD%,PO_Options,GRN_Options
0,Lifestyle,ALTERS,Men,Shorts,0.000000,701,0.000000,0.000000,0,0.000000,0.0,NaN,NaN,24,4
1,Lifestyle,ALTERS,Men,Track Pants,0.000000,123,0.000000,0.000000,0,0.000000,0.0,NaN,NaN,33,1
2,Lifestyle,ALTERS,Men,Tshirts,19255.108943,3214,11721.600156,900.778499,54,967.727418,48546.0,0.294208,0.603364,42,12
3,Lifestyle,ATH REDEEM,Men,Shorts,67381.237115,6764,43885.899334,3148.483998,180,3416.663240,182020.0,0.251260,0.629814,30,17
4,Lifestyle,ATH REDEEM,Men,Track Pants,283220.450311,9997,175124.398074,13294.314493,421,14312.282379,707379.0,0.284194,0.599620,42,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Yoga,NOWNESS,Women,Bra,10772.954158,3655,6287.924893,502.819500,25,546.246156,26575.0,0.318943,0.594621,33,15
118,Yoga,NOWNESS,Women,Shorts,0.000000,0,0.000000,0.000000,0,0.000000,0.0,NaN,NaN,12,0
119,Yoga,NOWNESS,Women,Tights,178509.734595,2340,106121.296591,8360.103995,352,8925.573117,449048.0,0.308682,0.602471,39,9
120,Yoga,NOWNESS,Women,Track Pants,58332.454921,1501,33995.395203,2737.466501,94,2884.340940,146506.0,0.320838,0.601843,24,6


In [152]:
#master_pivot.to_excel('COLLECTIONS.xlsx')

mrp_wise_master_pivot = pd.pivot_table(master,index=['sports','story','gender','article_type','mrp_bracket'],values=['GMV','Tax','Royalty','Total_MRP','Purchase_Price','Inventory','Sold_Qty'],
                              aggfunc=np.sum)
mrp_wise_master_pivot['GM%'] = (( mrp_wise_master_pivot['GMV'] - mrp_wise_master_pivot['Purchase_Price'] - mrp_wise_master_pivot['Tax'] - mrp_wise_master_pivot['Royalty'] ) / ( mrp_wise_master_pivot['GMV'] ))
mrp_wise_master_pivot['TD%'] = (( mrp_wise_master_pivot['Total_MRP'] - mrp_wise_master_pivot['GMV'] ) / ( mrp_wise_master_pivot['Total_MRP'] ))
mrp_wise_master_pivot = mrp_wise_master_pivot.reset_index()

mrp_wise_master_pivot = mrp_wise_master_pivot.reset_index()

def getPOQty(sports,story,gender,article,mrp_bracket):
    dump = po_dump[ (po_dump['sports'] == sports) & (po_dump['story'] == story) & (po_dump['gender'] == gender) & 
                    (po_dump['article_type'] == article) ]
    if len(dump) == 0:
        return 0
    else:
        count = len(dump['style_id'].unique())
        return count 


mrp_wise_master_pivot['PO_Qty'] = master_pivot.apply( lambda item :  getPOQty(item['sports'],item['story'],item['gender'],item['article_type'],item['mrp_bracket']), axis=1 )
mrp_wise_master_pivot['GRN_Qty'] = master_pivot.apply( lambda item :  getGRNQty(item['sports'],item['story'],item['gender'],item['article_type'],item['mrp_bracket']), axis=1 )


mrp_wise_master_pivot.to_excel('mrpwise.xlsx')

C:\Users\kunaljain.vc\AppData\Local\Temp\ipykernel_19952\3763486639.py:3: FutureWarning: The provided callable <function sum at 0x00000181D9C23D80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  mrp_wise_master_pivot = pd.pivot_table(master,index=['sports','story','gender','article_type','mrp_bracket'],values=['GMV','Tax','Royalty','Total_MRP','Purchase_Price','Inventory','Sold_Qty'],
